In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys
from html.parser import HTMLParser
from html.entities import name2codepoint
sns.set(color_codes=True)
import warnings
warnings.filterwarnings("ignore")                   
import nltk                                         
nltk.download('stopwords')
from nltk.corpus import stopwords                   
from nltk.stem import PorterStemmer  
LA = np.linalg

from sklearn.feature_extraction.text import CountVectorizer          
from sklearn.feature_extraction.text import TfidfVectorizer   
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from collections import defaultdict
from gensim.models.word2vec import Word2Vec                                  
import re

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chd415/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load training data text
def load_data(filename):
    load_file = pd.read_csv(filename,delimiter=',', header=0,
                        dtype={'name':str, 'lvl1':str, 'lvl2':str, 'lvl3':str, 'descrption':str, 'type':str})
    load_file.columns = ['id', 'name','lvl1','lvl2','lvl3','descrption','price','type']
    load_file.duplicated(subset=None, keep='first')
    load_file.set_index('id', inplace = True)
    load_file.head()
    return load_file
#print(len(train_file))
def load_label(filename):
    load_label = pd.read_csv(filename,delimiter=',', header=0)
    load_label.columns = ['id', 'score']
    load_label.duplicated(subset=None, keep='first')
    load_label.set_index('id', inplace = True)
    return load_label

In [3]:
'''
def map_mathod(column):
    values = []
    indexs = []
    mapping = {}
    index = 0
    for count in range(len(train_file)):
        value = train_file.get_value(count+1,column)
        if value in values and value != np.nan:
            continue
        values.append(value)
        indexs.append(len(values))
    for j in range(len(indexs)):
        mapping[values[j]] = indexs[j]
    mapping[np.nan] = 0.0
    return mapping
#train_file['lvl3'] = train_file['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
#mapping_lvl3 = map_mathod('lvl3')
#print(mapping_lvl3)
'''

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)


In [ ]:
'''
def text_embedding(column,vecsize):
    temp_X = column.astype(str)
    stop = set(stopwords.words('english'))
    temp =[]
    snow = nltk.stem.SnowballStemmer('english')

    for sentence in temp_X: 
        words = [snow.stem(word) for word in sentence.split(' ') if word not in stopwords.words('english')]   # Stemming and removing stopwords
        temp.append(sentence)

    count_vect = CountVectorizer(max_features=10000)
    bow_data = count_vect.fit_transform(temp)

    final_tf = temp
    tf_idf = TfidfVectorizer(max_features=10000)
    tf_data = tf_idf.fit_transform(final_tf)
    w2v_data = temp
    splitted = []
    for row in w2v_data: 
        splitted.append([word for word in row.split()])     #splitting words
    
    train_w2v = Word2Vec(splitted,min_count=1,size=vecsize, workers=4)
    
    avg_data = []
    for row in splitted:
        vec = np.zeros(vecsize, dtype=float)
        count = 0
        for word in row:
            try:
                vec += train_w2v[word]
                count += 1
            except:
                pass
        if (count == 0):
            avg_data.append(vec)
        else:
            avg_data.append(vec/count)

        
    tf_w_data = []
    tf_data = tf_data.toarray()
    i = 0
    for row in splitted:
        vec = [0 for i in range(vecsize)]
    
        temp_tfidf = []
        for val in tf_data[i]:
            if val != 0:
                temp_tfidf.append(val)
    
        count = 0
        tf_idf_sum = 0
        for word in row:
            try:
                count += 1
                tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
                vec += (temp_tfidf[count-1] * train_w2v[word])
            except:
                pass
        if (tf_idf_sum == 0):
            tf_w_data.append(vec)
        else:
            tf_w_data.append(vec/tf_idf_sum)
        i = i + 1
    
    return tf_w_data
'''

In [ ]:
'''
#test cell
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Normalizer

temp = load_data('train_data.csv')
price_X = temp['price'].as_matrix(columns=None).reshape(1, -1)

# scaler = StandardScaler()
# scaler.fit(price_X)
# outfile = scaler.transform(price_X)

# est = KBinsDiscretizer(n_bins=15, encode='onehot-dense', strategy='uniform')
# est.fit(price_X)
# outfile = est.transform(price_X)

transformer = Normalizer(norm='l2').fit(price_X)
outfile = transformer.transform(price_X)


# temp['price'] = temp['price']
# hist = temp['price'].hist(bins=15)

# price_X = temp['price'].clip(lower=0.).as_matrix(columns=None).tolist()
# price_X.sort()

#np.histogram(outfile,bins=10)

outfilelist = outfile.tolist()
outfilelist.sort()

file = open('temp.dat','w')
file.writelines(["%s\n" % item  for item in outfilelist])
file.close()
#print(outfile)
'''

In [ ]:
'''
#test cell

temp = pd.read_csv('train_data.csv',delimiter=',', header=0)
description_X = temp.descrption.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
#description_X.head()
tfidf_n = TfidfVectorizer(max_features=1000,stop_words = 'english')
tf_out = tfidf_n.fit_transform(description_X.astype('U')).toarray()
#lg_array = csr_matrix(tf_out, dtype=np.int8).toarray()
#lg_array = np.vstack( lg_array )
    
#U, s, Vh = LA.svd(lg_array, full_matrices=False)
#assert np.allclose(lg_array, np.dot(U, np.dot(np.diag(s), Vh)))

print(tf_out.shape)
'''

In [4]:
def clean_data(filename):
    
    filename['lvl1'] = filename['lvl1'].str.lower().replace('[^a-zA-Z]+',' ',regex=True)
    filename['lvl2'] = filename['lvl2'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['lvl3'] = filename['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['descrption'] = filename['descrption'].str.lower()
    filename['name'] = filename['name'].str.lower()

    '''
    
    mapping_lvl1 = map_mathod('lvl1')
    mapping_lvl2 = map_mathod('lvl2')
    mapping_lvl3 = map_mathod('lvl3')
    
    filename['lvl1'] = filename['lvl1'].map(mapping_lvl1)
    filename['lvl2'] = filename['lvl2'].map(mapping_lvl2)
    filename['lvl3'] = filename['lvl3'].map(mapping_lvl3)
    
    '''
    #clean up data for lvl 1&2&3 and type
    temp =  filename.drop(['price', 'descrption','name'], axis=1)
    outfile = MultiColumnLabelEncoder(columns = ['lvl1','lvl2','lvl3','type']).fit_transform(temp.astype(str))
#    outfile = outfile.as_matrix(columns=None).tolist()
    
    
    enc = preprocessing.OneHotEncoder()
    enc.fit(outfile)
    outfile = enc.transform(outfile).toarray()
    
    
    
    #normalize price
#     maxp = filename.price.max()
#     valuethred = 500.
#     filename['price'] = filename['price'].clip(lower=0.,upper=valuethred).div(valuethred,fill_value=0.)
#     hist = filename['price'].hist(bins=10)
#     #maxp
#     ld = filename.price.as_matrix(columns=None).tolist()

    price_X = filename['price'].as_matrix(columns=None).reshape(-1, 1)
    transformer = Normalizer(copy=True,norm='l2')
    ld = transformer.fit_transform(price_X)
    outfile = np.column_stack((outfile,ld))

    

    #clean up text
    description_X = filename.descrption.str.lower().replace('<li>','final ',regex=True).replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    count_descrption = description_X.str.count('final').fillna(0).tolist()
    outfile = np.column_stack((outfile,count_descrption))
    
    description_X = description_X.str.lower().replace('final ','',regex=True)
    filename['descrption'] = filename['descrption'].replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    descrption_Xstring = pd.Series(filename['descrption'].tolist()).astype(str)
    count_wordcount = descrption_Xstring.apply(lambda x: len(x.split(' ')))
    count_lettercount = descrption_Xstring.apply(lambda x: len(x))
    outfile = np.column_stack((outfile,count_wordcount))
    outfile = np.column_stack((outfile,count_lettercount))
    
#    tfidf_n = HashingVectorizer(n_features=2**20,stop_words = 'english')
    #tfidf_n = TfidfVectorizer(max_features=500,stop_words = 'english')
    tfidf_n = CountVectorizer(max_features=500,stop_words = 'english')
    lg = tfidf_n.fit_transform(description_X.astype('U')).toarray()
    lg_array = np.vstack( lg )
#    print(lg)
    
    U, s, Vh = LA.svd(lg_array, full_matrices=False)
    assert np.allclose(lg_array, np.dot(U, np.dot(np.diag(s), Vh)))
    s_new = s[:30]
    U_new = U[:, :30]
    new_lg = np.dot(U_new, np.diag(s_new))
    outfile = np.column_stack((outfile,new_lg))
    
    
    name_X = filename.name.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    filename['name'] = filename['name'].replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    name_Xstring = pd.Series(filename['name'].tolist()).astype(str)
    name_wordcount = name_Xstring.apply(lambda x: len(x.split(' ')))
    name_lettercount = name_Xstring.apply(lambda x: len(x))
    outfile = np.column_stack((outfile,name_wordcount))
    outfile = np.column_stack((outfile,name_lettercount))


#    tfidf_f = HashingVectorizer(n_features=2**20,stop_words = 'english')
    #tfidf_f = TfidfVectorizer(max_features=500,stop_words = 'english')
    tfidf_f = CountVectorizer(max_features=500,stop_words = 'english')
    lf = tfidf_f.fit_transform(name_X.astype('U')).toarray()
    lf_array = np.vstack( lf )

    Uf, sf, Vhf = LA.svd(lf_array, full_matrices=False)
    assert np.allclose(lf_array, np.dot(Uf, np.dot(np.diag(sf), Vhf)))
    sf_new = sf[:15]
    Uf_new = Uf[:, :15]
    new_lf = np.dot(Uf_new, np.diag(sf_new))
    outfile = np.column_stack((outfile,new_lf))
    
    '''
    enc = preprocessing.OneHotEncoder()
    enc.fit(outfile)
    outfile = enc.transform(outfile).toarray()
    '''

    return outfile

In [ ]:
'''
train_file = pd.read_csv('train_data.csv',delimiter=',', header=0, nrows=10)
train_file.columns = ['id', 'name','lvl1','lvl2','lvl3','descrption','price','type']
description_X = train_file.descrption.str.lower().replace('<li>','final ',regex=True).replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
count_descrption = description_X.str.count('final')
description_X = train_file.descrption.str.lower().replace('final ','',regex=True)
print(np.shape(count_descrption))

'''


In [5]:
train_file = load_data('train_data.csv')
test_file = load_data('test_data.csv')
combined_file = pd.concat([train_file,test_file])
cleaned_train = clean_data(combined_file)
train_score = load_label('train_label.csv')
np.shape(cleaned_train)

(36283, 305)

In [ ]:
'''
s_gra = np.abs(np.gradient(s))
slist = s.tolist()
file = open('svd_secrption.dat','w')
file.writelines(["%s\n" % item  for item in s_gra])
file.close()

import matplotlib.pyplot as plt
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_ylabel('Eign Values', color=color,fontweight='bold')
ax1.plot(slist, color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_xscale('log')


ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('Gradient of Eign Value', color=color,fontweight='bold')  # we already handled the x-label with ax1
ax2.plot(s_gra, color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_xscale('log')
ax2.set_yscale('log')
plt.grid()
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

fig.savefig('Eign_Values_descrption.png', dpi=fig.dpi)
'''

In [6]:
'''
# test cell
cleaned_train['descrption'] = temp['descrption']
'''
cleaned_train

array([[ 0.        ,  0.        ,  0.        , ..., -0.00184826,
         0.00700171, -0.00469088],
       [ 0.        ,  1.        ,  0.        , ...,  0.25687928,
        -0.2348867 , -0.04022385],
       [ 0.        ,  0.        ,  0.        , ...,  1.17822738,
        -0.37884069, -0.1530413 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.01015433,
        -0.00546545,  0.01432396],
       [ 0.        ,  0.        ,  0.        , ..., -0.037503  ,
        -0.12822085, -0.01662279],
       [ 0.        ,  0.        ,  0.        , ..., -0.03782297,
        -0.08829066, -0.1051121 ]])

In [ ]:
'''
pca = PCA(n_components=100, svd_solver='full')
cleaned_train_temp = pca.fit_transform(cleaned_train)

cleaned_train_min = np.min(cleaned_train_temp)
cleaned_train_out = cleaned_train_temp - (cleaned_train_min)*np.ones_like(cleaned_train_temp.size)
print(cleaned_train_out)
cleaned_train = cleaned_train_out
print(cleaned_train.shape)
'''

In [7]:
'''

def rearrange(cleaned_data):
    la = cleaned_data.lvl1.as_matrix(columns=None).tolist()
    lb = cleaned_data.lvl2.as_matrix(columns=None).tolist()
    lc = cleaned_data.lvl3.as_matrix(columns=None).tolist()

    X = la
    X = np.column_stack((X,lb))
    X = np.column_stack((X,lc))

    enc = preprocessing.OneHotEncoder()
    enc.fit(X)
    X = enc.transform(X).toarray()
    
    ld = cleaned_data.price.as_matrix(columns=None).tolist()
    le = cleaned_data.type.as_matrix(columns=None).tolist()

    X = np.column_stack((X,ld))
    X = np.column_stack((X,le))

   
    lf = cleaned_data.name.as_matrix(columns=None).tolist()
    lg = cleaned_data.descrption.as_matrix(columns=None).tolist()
    lg_array = np.vstack( lg )
    lf_array = np.vstack( lf )

    U, s, Vh = LA.svd(lg_array, full_matrices=False)
    assert np.allclose(lg_array, np.dot(U, np.dot(np.diag(s), Vh)))

# only use U \cdot s
    
#    s[8:] = 0.
#    new_lg = np.dot(U, np.dot(np.diag(s), Vh))
    s_new = s[:5]
    U_new = U[:, :5]
    new_lg = np.dot(U_new, np.diag(s_new))
    lg_min = np.min(new_lg)
    lg_out = new_lg  - (lg_min-2)*np.ones_like(new_lg.size)

    Uf, sf, Vhf = LA.svd(lf_array, full_matrices=False)
    assert np.allclose(lf_array, np.dot(Uf, np.dot(np.diag(sf), Vhf)))

#    sf[5:] = 0.
#    new_lf = np.dot(Uf, np.dot(np.diag(sf), Vhf))
    sf_new = sf[:5]
    Uf_new = Uf[:, :5]
    new_lf = np.dot(Uf_new, np.diag(sf_new))
    lf_min = np.min(new_lf)
    lf_out = new_lf  - (lf_min-2)*np.ones_like(new_lf.size)

    
    X = np.column_stack((X,lf_out))
    X = np.column_stack((X,lg_out))
    X = X.tolist()
   
    return X,lf_min,lg_min
'''
    
    #print(len(X))
X = cleaned_train#rearrange(cleaned_train)
w,b = np.shape(np.array(X))
print(np.shape(np.array(X)))
Y = train_score.score.as_matrix(columns=None).tolist()
b

(36283, 305)


305

In [8]:
X = cleaned_train[:18141]
XX = cleaned_train[18141:]
Y = train_score.score.as_matrix(columns=None).tolist()
print(np.size(Y))
print(np.shape(X))
print(np.shape(XX))


18141
(18141, 305)
(18142, 305)


In [9]:
from sklearn.utils import shuffle
X, Y = shuffle(X, Y)
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, test_size=0.20, random_state=0)


#maxlen = b
#X_train = sequence.pad_sequences(X_train, maxlen=maxlen, dtype='float32')
#X_validation = sequence.pad_sequences(X_validation, maxlen=maxlen, dtype='float32')
print(X_train[1400].size)

305


In [10]:
import os
import time
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
import pandas as pd
import random
import math
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2

from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
#from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import BaggingClassifier  
from sklearn.model_selection import cross_val_score

In [11]:
# Linear Regression Classifier
def linear_regression_classifier(train_x, train_y):
    model = linear_model.LinearRegression()
    model.fit(train_x, train_y)
    return model
 
# Multinomial Naive Bayes Classifier
def naive_bayes_classifier(train_x, train_y):
    model = MultinomialNB()

    param_grid = {'alpha': [math.pow(10,-i) for i in range(11)]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = MultinomialNB(alpha = best_parameters['alpha'])  
    model.fit(train_x, train_y)
    return model
 
 
# KNN Classifier
def knn_classifier(train_x, train_y):
    model = KNeighborsClassifier()

    param_grid = {'n_neighbors': list(range(1,21))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = KNeighborsClassifier(n_neighbors = best_parameters['n_neighbors'], algorithm='kd_tree')

    bagging = BaggingClassifier(model, max_samples=0.5, max_features=1 )
    bagging.fit(train_x, train_y)
    return bagging
 
 
# Logistic Regression Classifier
def logistic_regression_classifier(train_x, train_y):
    model = LogisticRegression(penalty='l2')
    model.fit(train_x, train_y)
    return model
 
 
# Random Forest Classifier
def random_forest_classifier(train_x, train_y):
    model = RandomForestClassifier()

    param_grid = {'n_estimators': list(range(1,21))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = RandomForestClassifier(n_estimators = best_parameters['n_estimators'])
    
    model.fit(train_x, train_y)
    return model
 
 
# Decision Tree Classifier
def decision_tree_classifier(train_x, train_y):
    model = tree.DecisionTreeClassifier()
    model.fit(train_x, train_y)

    bagging = BaggingClassifier(model, max_samples=0.5, max_features=1 )
    bagging.fit(train_x, train_y)
    
    return bagging
 
 
# GBDT(Gradient Boosting Decision Tree) Classifier
def gradient_boosting_classifier(train_x, train_y):
    model = GradientBoostingClassifier()
    
    model = RandomForestClassifier()

    param_grid = {'n_estimators': list(range(100,300,10))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = RandomForestClassifier(n_estimators = best_parameters['n_estimators'])

    model.fit(train_x, train_y)
    return model

# SVM Classifier
def svm_classifier(train_x, train_y):
    model = SVC(kernel='linear', probability=True)
    model.fit(train_x, train_y)
    return model
 
# SVM Classifier using cross validation
def svm_cross_validation(train_x, train_y):
    model = SVC(kernel='linear', probability=True)
    param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    #for para, val in best_parameters.items():
        #print para, val
    model = SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], probability=True)
    model.fit(train_x, train_y)
    return model

def feature_select(x,y):
    clf = ExtraTreesClassifier()
    clf = clf.fit(x, y)
    model = SelectFromModel(clf, prefit=True)
    return model

In [13]:
# just for my own record


if __name__ == '__main__': 
    thresh = 0.5    
    
    test_classifiers = ['KNN','LR','RF','DT']    
    classifiers = {
                   'KNN':knn_classifier,
                    'LR':logistic_regression_classifier,
                    'RF':random_forest_classifier,
                    'DT':decision_tree_classifier,
    }
    
    '''
    test_classifiers = ['SVC']    
    classifiers = {'SVC':svm_cross_validation}
    ''' 
    
    print('reading training and testing data...')    

    select_model = feature_select(X_train, y_train)
    X_train = select_model.transform(X_train)
    X_validation = select_model.transform(X_validation)

    result = []
    
    '''
    start_time = time.time()    
    model = classifiers[classifier](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    regressor = DecisionTreeRegressor()
    regressor.fit(train_x, train_y)
    predict = model.predict(X_validation)
    '''

        
    for classifier in test_classifiers:    
        print('******************* %s ********************' % classifier)    
        start_time = time.time()    
        model = classifiers[classifier](X_train, y_train)   
        print('training took %fs!' % (time.time() - start_time))    
        predict = model.predict(X_validation)

        precision = metrics.precision_score(y_validation, predict)    
        recall = metrics.recall_score(y_validation, predict)    
        accuracy = metrics.accuracy_score(y_validation, predict)    
        print('accuracy: %.2f%%' % (100 * accuracy))
        logloss = metrics.log_loss(y_validation, predict)
        print('loss: %.2f' % (logloss))

        scores = cross_val_score(model, X_train, y_train)
#        print(scores)

reading training and testing data...
******************* KNN ********************
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   16.0s finished


training took 16.141019s!
accuracy: 68.92%
loss: 10.74
******************* LR ********************
training took 0.080522s!
accuracy: 78.23%
loss: 7.52
******************* RF ********************
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   15.2s finished


training took 16.412083s!
accuracy: 79.72%
loss: 7.00
******************* DT ********************
training took 0.329029s!
accuracy: 68.45%
loss: 10.90


In [14]:
X_train
#X_train, y_train = X,Y

array([[ 0.        ,  1.        ,  0.        , ..., -0.13150647,
        -0.2752178 , -0.18125084],
       [ 0.        ,  0.        ,  1.        , ...,  0.09084801,
         0.06534211, -0.06657633],
       [ 0.        ,  1.        ,  0.        , ..., -0.05854416,
        -0.09125824, -0.03681251],
       ...,
       [ 1.        ,  1.        ,  0.        , ...,  0.80953534,
        -0.00779874, -0.7293216 ],
       [ 0.        ,  0.        ,  1.        , ...,  0.0056889 ,
        -0.05862812,  0.13158637],
       [ 0.        ,  0.        ,  1.        , ..., -0.02963569,
         0.0087225 ,  0.02193457]])

In [15]:
#test cnn model

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

# Embedding
max_features = 5000
maxlen = b
embedding_size = 128

# Convolution
kernel_size = 5
filters = 32
pool_size = 3

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 5

'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''

print('Loading data...')
X_train = np.asarray(np.abs(X_train))
X_validation = np.asarray(np.abs(X_validation))
print(len(X_train), 'train sequences')
print(len(X_validation), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_validation = sequence.pad_sequences(X_validation, maxlen=maxlen, padding='post')
print('x_train shape:', X_train.shape)
print('x_test shape:', X_validation.shape)

print('Build model...')

cnnmodel = Sequential()
cnnmodel.add(Embedding(max_features, embedding_size,input_length=maxlen))
#model.add(Dense(32, activation='relu', input_dim=100))
cnnmodel.add(Dropout(0.5))
cnnmodel.add(Dense(32, activation='relu', input_dim=64))
cnnmodel.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
cnnmodel.add(MaxPooling1D(pool_size=pool_size))
cnnmodel.add(LSTM(lstm_output_size))
cnnmodel.add(Dense(1))
cnnmodel.add(Activation('sigmoid'))

cnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


Using TensorFlow backend.


Loading data...
14512 train sequences
3629 test sequences
Pad sequences (samples x time)
x_train shape: (14512, 305)
x_test shape: (3629, 305)
Build model...


In [16]:
X_train = cleaned_train[:18141]
X_test = cleaned_train[18141:]
y_train = train_score.score.as_matrix(columns=None).tolist()

In [17]:
if __name__ == '__main__': 
    thresh = 0.5    
     
    test_classifiers = ['LR','RF','GBC','DT']    
    classifiers = {
                   'LR':logistic_regression_classifier,
                   'RF':random_forest_classifier,
                   'GBC':gradient_boosting_classifier,
                   'DT':decision_tree_classifier
    }
        
    print('reading training and testing data...')    

    select_model = feature_select(X_train, y_train)
    X_train = select_model.transform(X_train)
    X_test = select_model.transform(X_test)

    result = []
    
    start_time = time.time()
    model = classifiers['LR'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_lr = model.predict_proba(X_test)[:,1]
    print('predict finished')
    
    model = classifiers['RF'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))
    regr = RandomForestRegressor(max_depth=2, max_features=1)
    regr.fit(X_train, y_train)
    Y_predict_rf = regr.predict(X_test)
    print('predict finished')
    
    model = classifiers['GBC'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_gbc = model.predict_proba(X_test)[:,1]
    print('predict finished')
    
    model = classifiers['DT'](X_train, y_train)  
    print('training took %fs!' % (time.time() - start_time))
    regressor = DecisionTreeRegressor(max_depth=2) 
    regressor.fit(X_train, y_train)
    Y_predict_dt = regressor.predict(X_test)
    print('predict finished')

reading training and testing data...
training took 1.012821s!
predict finished
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   43.9s finished


training took 49.043003s!
predict finished
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 12.7min finished


training took 838.457982s!
predict finished
training took 840.440748s!
predict finished


In [18]:
#cnn test
X_train = cleaned_train[:18141]
X_test = cleaned_train[18141:]
y_train = train_score.score.as_matrix(columns=None).tolist()
X_train = np.asarray(X_train)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')


cnnmodel.fit(X_train, y_train, batch_size=128, epochs=8)

Y_predict_cnn = cnnmodel.predict(X_test, verbose=0)
Y_predict_cnn = np.squeeze(Y_predict_cnn, axis=1)

Epoch 1/8


InvalidArgumentError: indices[124,266] = -2 is not in [0, 5000)
	 [[Node: embedding_1/embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training/Adam/Assign_2"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_1/embeddings/read, embedding_1/Cast, training/Adam/gradients/embedding_1/embedding_lookup_grad/concat/axis)]]

In [26]:
#nn train
X_train = cleaned_train[:18141]
X_test = cleaned_train[18141:]
y_train = train_score.score.as_matrix(columns=None).tolist()
X_array = np.asarray(X_train)
Y_array = np.asarray(y_train)
Xtest_array = np.asarray(X_test) 
import warnings
warnings.filterwarnings("ignore")   
# Create your first MLP in Keras
from keras.models import Sequential
from keras.layers import Dense


# fix random seed for reproducibility
np.random.seed(7)

# split into input (X) and output (Y) variables
X = X_array
Y = Y_array
# create model
nnmodel = Sequential()
nnmodel.add(Dense(100, input_dim=b, activation='relu'))
nnmodel.add(Dense(100, activation='relu'))
nnmodel.add(Dense(100, activation='relu'))
nnmodel.add(Dense(100, activation='relu'))
nnmodel.add(Dense(1, activation='sigmoid'))
# Compile model
nnmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
nnmodel.fit(X, Y, epochs=20, batch_size=150)
# evaluate the model
scores = nnmodel.evaluate(X, Y)
print("\n%s: %.2f%%" % (nnmodel.metrics_names[1], scores[1]*100))
Y_predict_nn = nnmodel.predict(Xtest_array, verbose=0)
Y_predict_nn = np.squeeze(Y_predict_nn, axis=1)

Epoch 1/20
18141/18141 [==============================] - 1s 58us/step - loss: 0.6471 - acc: 0.6958
Epoch 2/20
18141/18141 [==============================] - 0s 26us/step - loss: 0.5038 - acc: 0.7681
Epoch 3/20
18141/18141 [==============================] - 1s 29us/step - loss: 0.4774 - acc: 0.7781
Epoch 4/20
18141/18141 [==============================] - 0s 26us/step - loss: 0.4535 - acc: 0.7962
Epoch 5/20
18141/18141 [==============================] - 0s 25us/step - loss: 0.4408 - acc: 0.8025
Epoch 6/20
18141/18141 [==============================] - 0s 25us/step - loss: 0.4482 - acc: 0.7946
Epoch 7/20
18141/18141 [==============================] - 0s 25us/step - loss: 0.4371 - acc: 0.8002
Epoch 8/20
18141/18141 [==============================] - 0s 26us/step - loss: 0.4372 - acc: 0.8034
Epoch 9/20
18141/18141 [==============================] - 0s 27us/step - loss: 0.4265 - acc: 0.8072
Epoch 10/20
18141/18141 [==============================] - 1s 31us/step - loss: 0.4259 - acc: 0.8083

In [27]:
Y_predict_nn

array([0.94319683, 0.883781  , 0.42291856, ..., 0.81127375, 0.3868661 ,
       0.9203107 ], dtype=float32)

In [21]:
Y_predict_cnn

NameError: name 'Y_predict_cnn' is not defined

In [28]:
Y_predict_lr

array([0.83165771, 0.65365511, 0.13319898, ..., 0.62160562, 0.31380171,
       0.89526246])

In [29]:
Y_predict_rf

array([0.72274931, 0.67831972, 0.68261472, ..., 0.69777779, 0.67477614,
       0.67595352])

In [30]:
Y_predict_gbc

array([0.93846154, 0.80769231, 0.48461538, ..., 0.69230769, 0.46153846,
       0.74615385])

In [25]:
Y_predict_dt

array([0.68045831, 0.5925335 , 0.5925335 , ..., 0.88458665, 0.26255507,
       0.88458665])

In [32]:
Y_predict = []
for i in range(len(Y_predict_rf)):
    Y_predict.append(( Y_predict_rf[i] + Y_predict_lr[i] + Y_predict_gbc[i] + Y_predict_nn[i] ) / 4.0)

In [33]:
Y_predict

[0.8590163466297449,
 0.7558620363130346,
 0.43083691027233806,
 0.8804154103492244,
 0.8685857363349854,
 0.803982722191195,
 0.3702047910112911,
 0.8821336861563667,
 0.7060969727764501,
 0.8419911805000246,
 0.845600119417621,
 0.5769959365445285,
 0.6638917911539857,
 0.49603111755952123,
 0.9041633913442157,
 0.4669674019164296,
 0.8494922424850394,
 0.2939765106866558,
 0.8479966208965481,
 0.7023654367491607,
 0.8926189427318618,
 0.8075674221410716,
 0.8900378071554869,
 0.803539955430323,
 0.8879795677536544,
 0.7362218941551693,
 0.6231352368319243,
 0.9084247127671244,
 0.8264634977099837,
 0.8813748464462534,
 0.8996368760280984,
 0.7652600087000799,
 0.8697013559822653,
 0.790142160404932,
 0.711038400553345,
 0.6474023492348753,
 0.5300532425972011,
 0.8258774456968576,
 0.558745779128343,
 0.8427897542676432,
 0.7411771736784677,
 0.8997632682046338,
 0.8622091052427368,
 0.7542736762625616,
 0.5775285248246292,
 0.8842282264033381,
 0.6966422561149436,
 0.34175065135574

In [ ]:
Y_predict_temp = []
for i in range(len(Y_predict_rf)):
    Y_predict_temp.append((Y_predict_rf[i] + Y_predict_gbc[i]) / 2.0)

In [ ]:
Y_predict_temp 

In [34]:
temp_score = load_label('submission.csv')
submit_score  = temp_score
submit_score['score'] = Y_predict
submit_score.to_csv('predict_result_cnn.csv')

In [ ]:
temp_score = load_label('submission.csv')
submit_score  = temp_score
submit_score['score'] = Y_predict_temp
submit_score.to_csv('predict_result_ncnn.csv')

In [ ]:

AA = load_label('predict_result_488.csv')
aa = AA.score.as_matrix(columns=None).tolist()
BB = load_label('predict_result_495.csv')
bb = BB.score.as_matrix(columns=None).tolist()
CC = load_label('predict_result_509.csv')
cc = CC.score.as_matrix(columns=None).tolist()
DD = load_label('predict_result_cnn.csv')
dd = DD.score.as_matrix(columns=None).tolist()

comb = []
for i in range(len(cc)):
    comb.append((aa[i] + bb[i] + cc[i] + dd[i]) / 4.0)

#comb


In [ ]:

temp_score = load_label('submission.csv')
submit_score  = temp_score
submit_score['score'] = dd
submit_score.to_csv('combined.csv')


In [ ]:
table = pd.pivot_table(train_file,values = 'price', index=['lvl1','lvl2','lvl3'],columns=['type'],aggfunc=[min, max, np.mean])
table